In [31]:
from src.capstone.cbf import CBF, InfeasibilityError
from src.capstone.settings import Env

from double_integrator import DoubleIntegratorEnv
import gymnasium as gym

from bound_propagation.polynomial import Pow
from bound_propagation.linear import FixedLinear

import numpy as np

import torch
import torch.nn as nn

import matplotlib.pyplot as plt
from tqdm import tqdm

$$B = \begin{bmatrix}
\frac{1}{2}\Delta t^2 & 0\\
0 & \frac{1}{2}\Delta t^2\\
\Delta t & 0\\
0 & \Delta t
\end{bmatrix}$$

$$Q = B B^T = \begin{bmatrix}
\frac{1}{2}\Delta t^2 & 0\\
0 & \frac{1}{2}\Delta t^2\\
\Delta t & 0\\
0 & \Delta t
\end{bmatrix} 

\begin{bmatrix}
\frac{1}{2}\Delta t^2 & 0 & \Delta t & 0\\
0 & \frac{1}{2}\Delta t^2 & 0 & \Delta t
\end{bmatrix} 

= \begin{bmatrix}
\frac{1}{4}\Delta t^4 & 0 & \frac{1}{2}\Delta t^3 & 0\\
0 & \frac{1}{4}\Delta t^4 & 0 & \frac{1}{2}\Delta t^3\\
\frac{1}{2}\Delta t^3 & 0 & \Delta t^2 & 0\\
0 & \frac{1}{2}\Delta t^3 & 0 & \Delta t^2
\end{bmatrix}$$

In [32]:
class IntegratorNoise(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        dt = 0.05
        self.cov = np.array([[0.25 * dt**4, 0., 0.5 * dt**3, 0.],
                           [0., 0.25 * dt**4, 0., 0.5 * dt**3],
                           [0.5 * dt**3, 0., dt**2, 0.],
                           [0., 0.5 * dt**3, 0., dt**2]])

    def step(self, action):
        state, _, _, truncated, _ = self.env.step(action)
        noise = np.random.multivariate_normal(mean=np.zeros(4), cov=self.cov)

        return (state + noise).astype(np.float32), 0., False, truncated, {}

    def reset(self, seed=None):
        return self.env.reset(seed=seed)

In [33]:
class NNDM(nn.Sequential):
    # input [x pos, y pos, x speed, y speed, ux, uy]
    
    def __init__(self):
        super(NNDM, self).__init__()
        self.dt = 0.05
        
        layer = nn.Linear(6, 4, bias=False)
        weights = torch.tensor([[1., 0., self.dt, 0., ((self.dt**2)/2), 0], 
                                [0., 1., 0., self.dt, 0., ((self.dt**2)/2)], 
                                [0., 0., 1., 0., self.dt, 0.], 
                                [0., 0., 0., 1., 0., self.dt]])
        layer.weight = nn.Parameter(weights)
        
        super(NNDM, self).__init__(layer)

nndm = NNDM()

In [34]:
class Agent(nn.Module):
    def __init__(self):
        super(Agent, self).__init__()
        
        self.layer = nn.Linear(2, 2)
        
        # dummy output of u=0
        self.layer.weight = nn.Parameter(torch.tensor([[0., 0., 0., 0.], [0., 0., 0., 0.]]))
        self.layer.bias = nn.Parameter(torch.tensor([50., 0.]))
    
    def forward(self, x):
        return self.layer(x)
    
    def select_action(self, x, exploration=False):
        if exploration:
            raise ValueError('This model is not implemented for exploration')
        else:
            return self.forward(x)

In [35]:
policy = Agent()

In [36]:
gym.register('DoubleIntegrator', DoubleIntegratorEnv, max_episode_steps=100)

C:\Users\ruben\PycharmProjects\Capstone\venv\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment DoubleIntegrator already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [53]:
class DoubleIntegrator(Env):
    def __init__(self):
        env = gym.make('DoubleIntegrator')
        
        self.is_discrete = False
        
        self.settings = {
            'noise': [],
            'max_frames': 999
        }
        
        self.h_function = nn.Sequential(
            FixedLinear(
                torch.tensor([
                    [1., 0, 0, 0],
                    [0, 1, 0, 0]
                ]),
                torch.tensor([0., 0.])
            ),
            Pow(2),
            FixedLinear(
                torch.tensor([
                    [-1 / 0.5 ** 2, 0],
                    [0, -1 / 0.5 ** 2]
                ]),
                torch.tensor([1., 1.])
            )
        )

        self.h_ids = [0, 1]
        self.std = [0.05**2/2, 0.05**2/2]
        self.env = IntegratorNoise(env)

In [54]:
env = DoubleIntegrator()

In [55]:
action = [50., 0.]

In [60]:
alpha = 0.9875

cbf = CBF(env, nndm, policy,
          alpha=[alpha, alpha],
          delta=[0., 0.],
          no_action_partitions=16,
          no_noise_partitions=5,
          stochastic=True)

In [61]:
"""evaluator = Evaluator(env, cbf)
f, h = evaluator.mc_simulate(policy, 10, cbf_enabled=True, seed=[0., 0., 0., 0.])"""

'evaluator = Evaluator(env, cbf)\nf, h = evaluator.mc_simulate(policy, 10, cbf_enabled=True, seed=[0., 0., 0., 0.])'

In [62]:
def plot_trajectories(num_agents, start_state, cbf_enabled=False):
    # plot the trajectories of the system, starting from the origin

    if start_state is None:
        start_state = [0., 0., 0., 0.]
    states = []

    for _ in tqdm(range(num_agents)):
        state_list = []
        state, _ = env.env.reset(seed=start_state)
        state = torch.tensor(state).unsqueeze(0)

        done = False

        while not done:   
            state_list.append(state.squeeze())

            # try cbf action - if cbf disabled or no safe actions available, just follow agent policy
            if cbf_enabled:
                try:
                    action = cbf.safe_action(state)
                except InfeasibilityError:
                    terminated = True
                    done = True
                    break
            else:
                action = policy.select_action(state, exploration=False)

            state, reward, terminated, truncated, _ = env.env.step(action.squeeze().detach().numpy())
            state = torch.tensor(state).unsqueeze(0)

            if torch.any(env.h_function(state.unsqueeze(0)) < 0).item():
                terminated = True

            done = terminated or truncated

        states.append(np.array(state_list))

    env.env.close()
    
    print(states)
    
    
    x_circle = np.linspace(-0.5, 0.5, 1001)
    
    y_circle_upper = [np.sqrt((0.5)**2 - x**2) for x in x_circle]
    y_circle_lower = [-np.sqrt((0.5)**2 - x**2) for x in x_circle]

    plt.plot(x_circle, y_circle_upper, color ="black")
    plt.plot(x_circle, y_circle_lower, color = "black")
    plt.fill_between(x_circle, y_circle_upper, y_circle_lower, color='g', alpha=0.3)
    x_axis = np.linspace(-0.5, 0.5, 3)
    y_axis = np.linspace(-0.5, 0.5, 3)

    plt.xticks(x_axis)
    plt.yticks(y_axis)

    plt.title("Trajectories")
    plt.xlabel("x")
    plt.ylabel("y")

    for trajectory in states:
        # https://stackoverflow.com/questions/21519203/plotting-a-list-of-x-y-coordinates 
        x, y, x_dot, y_dot = zip(*trajectory)
        plt.plot(x, y, c='b', alpha=0.1)


    plt.savefig("Double_integrator_trajectory_plot.png")
    plt.show()



In [59]:
plot_trajectories(1, [0., 0., 0., 0.], cbf_enabled=True)

  0%|          | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
def p_u_theoretical(xg, yg):
    res = np.zeros([xg.shape[1], yg.shape[0]])
    
    x_y = np.dstack((xg, yg))
    gridpoints = [point for row in x_y for point in row]
    h_values = [env.h_function(torch.tensor(point, dtype=torch.float32)) for point in gridpoints]
     
    idx = 0  # could have used enumerate(gridpoints), but this would disallow the tqdm progress bar
    for point in tqdm(gridpoints):
        p_u = 1 - (env.h_function(torch.tensor(point, dtype=torch.float32)).item() * alpha ** 100)
        
        i, j = idx % x_y.shape[1], idx // x_y.shape[0]
        res[j, i] = p_u
        
        idx += 1
    
    clipped_res = np.clip(res, 0., 1.)  
    return clipped_res

In [49]:
def p_u_experimental(xg, yg, num_agents=500):
    res = np.zeros([xg.shape[1], yg.shape[0]])

    x_y = np.dstack((xg, yg))
    gridpoints = [point for row in x_y for point in row]
    
    idx = 0
    for point in tqdm(gridpoints):
        end_frames, _ = evaluator.mc_simulate(policy, num_agents, cbf_enabled=True, seed=list(point), progress_bar=False)
        p_u = sum(f <= 100 for f in end_frames)/len(end_frames) if len(end_frames) > 0 else 0.
        
        i, j = idx % x_y.shape[1], idx // x_y.shape[0]
        res[i, j] = p_u
        
        idx += 1     
    return res

In [50]:
#Safety value h(x) plot

In [51]:
#Fraction unsafe plot